In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from tqdm import tqdm
import pytz
from matplotlib import pyplot as plt
import warnings
from sklearn.metrics import accuracy_score
from datetime import date
import random
from scipy.optimize import minimize

warnings.simplefilter(action='ignore', category=FutureWarning)
tz = pytz.timezone("America/New_York")

In [ ]:
yt_df = pd.read_csv('/content/drive/MyDrive/FYP - Experiment Evaluation/youtube_pred_2024.csv')

date_new = []
for idx,row in yt_df.iterrows():
    arr = row['Upload date'].split(" ")
    date = arr[0]
    date_new.append(date)

date_new_df=pd.DataFrame(date_new)
date_new_df.columns = ['date_new']
yt_df = yt_df.join(date_new_df)
yt_df['date_new'] = pd.to_datetime(yt_df['date_new'])
yt_df['Upload date'] = yt_df['date_new'].apply(lambda x: x.date())
yt_df.pop('date_new')
yt_df

In [ ]:
def price_moving_level (symbol,date,window,threshold_price):

    stock = yf.Ticker(symbol)
    start_date =  date - datetime.timedelta(days=1)
    end_date = date + datetime.timedelta(days=30)

    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    df_stock = stock.history(start = start_date, end =end_date).reset_index()
    p = -1

    if df_stock.empty:
        return p

    else:
        df_stock["Date"] = pd.to_datetime(df_stock["Date"])
        df_stock['date'] = df_stock['Date'].dt.date
        c0 = df_stock[df_stock['date']==date]['Close'].values

        if c0.size ==0:
            return p

        else:
            next_idx = df_stock[df_stock['date']==date].index + window
            c1=df_stock.loc[next_idx]['Close'].values

            c = (c1[0]-c0[0] )/c0[0]

            if c > threshold_price:
                p = 2
            elif c < -threshold_price :
                p = 0
            else:
                p = 1

            return p


def evaluate_yt(channel,window,threshold_price):
    df_all = yt_df[yt_df['Channel']==channel]
    dates = df_all['Upload date'].value_counts()

    sen_arr = []
    price_arr = []
    ticker_arr = []

    for date, cnt_date in tqdm(dates.items()):
        df_today = df_all[df_all['Upload date'] == date]
        symbols= df_today['Symbol'].value_counts()

        for sym, cnt_sym in symbols.items():
            df_sen = df_today[df_today['Symbol']==sym]
            sen_sum = df_sen['pred_sen'].sum()
            price_level = price_moving_level(sym,date,window,threshold_price)

            if price_level >=0:
                sen_avg = sen_sum /cnt_sym
                sen_arr.append(round(sen_avg))
                price_arr.append(price_level)
                ticker_arr.append(sym)

    acc = accuracy_score(price_arr,sen_arr)
    return acc, price_arr, sen_arr

In [ ]:
yt_eval_df =  pd.DataFrame(columns = ['channel', 'time_window', 'acc'])
channels = yt_df['Channel'].value_counts()

for i in range(10):
  for cha, cnt_cha in authors.items():
    acc, price_arr, sen_arr = evaluate_yt(cha,i+1,0.01)
    new_row = {'channel':cha,'time_window':i+1, 'accuracy':acc}
    yt_eval_df = pd.concat([yt_eval_df, pd.DataFrame([new_row])], ignore_index = True)

yt_eval_df